In [16]:
##### fit Zeeman splitting spectrum using the minimize function of scipy
#Jan/29/2021, Tao-Chung Ching, chingtaochung@gmail.com
#1 wcm background, 3 cnm in tau, 1 absorption foreground in tau
#fit tau, height, width, center of the components

import matplotlib.pyplot as plt
import numpy as np
import sys

import matplotlib.style
import matplotlib as mpl
mpl.style.use('classic')

veltmp = np.loadtxt("L1544_vel.txt")
stk_Itmp = np.loadtxt("L1544_I_spec.txt")
stk_Vtmp = np.loadtxt("L1544_V_spec.txt")
vel = veltmp[::-1]
stk_I = stk_Itmp[::-1]
stk_V = stk_Vtmp[::-1]

#linear fit to remove baseline
x = np.concatenate((vel[0:50], vel[-50:-1]), axis=0)
y = np.concatenate((stk_I[0:50], stk_I[-50:-1]), axis=0)
z = np.polyfit(x, y, 1)
stk_fit = stk_I-(z[0]*vel+z[1])

y = np.concatenate((stk_V[0:50], stk_V[-50:-1]), axis=0)
z = np.polyfit(x, y, 1)
stk_Vfit = stk_V-(z[0]*vel+z[1])
hanning = np.array([0.25,0.5,0.25]) 
stk_Vfit = np.convolve(stk_Vfit, hanning, mode='same')

fig = plt.figure() 

###initial guess, 0 hinsa, 1,2,3 cnm, w wnm###
t0 = 0.3
c0 = 6.96
w0 = 0.36

h1 = 90.
t1 = 0.8
c1 = 8.1
w1 = 1.9

h2 = 116.
t2 = 0.8
c2 = -0.4
w2 = 2.4

h3 = 135.
t3 = 10.4
c3 = 4.4
w3 = 2.

hw = 46. #136.8
fw = 0.
cw = 2.6
ww = 6.4

###boundary of fit###
t0b = [0.,1.]
c0b = [6.7,7.2]
w0b = [0.1,3.]

hb = [0.,1000.]
tb = [0.,100.]
cb = [-20.,20.]
wb = [0.1,10.]

hwb = [0.,200.]
cwb = [-20.,20.]
wwb = [0.,10.]

def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.))) #/(sig*np.sqrt(2*np.pi))

def fitwcca(vel, h1, t1, c1, w1, h2, t2, c2, w2, h3, t3, c3, w3, hw, cw ,ww, t0, c0, w0):
    tau0 = t0*gaussian(vel, c0, w0)
    tau1 = t1*gaussian(vel, c1, w1)
    tau2 = t2*gaussian(vel, c2, w2)
    tau3 = t3*gaussian(vel, c3, w3)
    tau = tau0+tau1+tau2+tau3
    Tc1 = h1*(1.-np.exp(-tau1))*np.exp(-tau0)
    Tc2 = h2*(1.-np.exp(-tau2))*np.exp(-tau1)*np.exp(-tau0)
    Tc3 = h3*(1.-np.exp(-tau3))*np.exp(-tau2)*np.exp(-tau1)*np.exp(-tau0)
    Tc = Tc1+Tc2+Tc3
    Tw = (fw+(1.-fw)*np.exp(-tau))*hw*gaussian(vel, cw, ww)
    model = Tc+Tw
    return model 

from scipy.optimize import curve_fit
popt,pcov = curve_fit(fitwcca,vel,stk_fit,p0=[h1, t1, c1, w1, h2, t2, c2, w2, h3, t3, c3, w3 ,hw, cw ,ww, t0, c0, w0],
                      bounds=([hb[0],tb[0],cb[0],wb[0],hb[0],tb[0],cb[0],wb[0],hb[0],tb[0],cb[0],wb[0],hwb[0],cwb[0],wwb[0],t0b[0],c0b[0],w0b[0]],
                              [hb[1],tb[1],cb[1],wb[1],hb[1],tb[1],cb[1],wb[1],hb[1],tb[1],cb[1],wb[1],hwb[1],cwb[1],wwb[1],t0b[1],c0b[1],w0b[1]]))

h1 = popt[0]
t1 = popt[1]
c1 = popt[2]
w1 = popt[3]
h2 = popt[4]
t2 = popt[5]
c2 = popt[6]
w2 = popt[7]
h3 = popt[8]
t3 = popt[9]
c3 = popt[10]
w3 = popt[11]
hw = popt[12]
cw = popt[13]
ww = popt[14]
t0 = popt[15]
c0 = popt[16]
w0 = popt[17]

perr = np.sqrt(np.diag(pcov))
print perr
rnum = 2
print 'h1 =',h1,perr[0],'t1 =',t1,perr[1],'c1 =',c1,perr[2],'w1 =',w1,perr[3]
print 'h2 =',h2,perr[4],'t2 =',t2,perr[5],'c2 =',c2,perr[6],'w2 =',w2,perr[7]
print 'h3 =',h3,perr[8],'t3 =',t3,perr[9],'c3 =',c3,perr[10],'w3 =',w3,perr[11]
print 'hw =',hw,perr[12],'fw =',fw,'cw =',cw,perr[13],'ww =',ww,perr[14]
print 't0 =',t0,perr[15],'c0 =',c0,perr[16],'w0 =',w0,perr[17]

print 'h1 =',h1.round(2),perr[0].round(2),'t1 =',t1.round(2),perr[1].round(2),'c1 =',c1.round(2),perr[2].round(2),'w1 =',w1.round(2),perr[3].round(2)
print 'h2 =',h2.round(2),perr[4].round(2),'t2 =',t2.round(2),perr[5].round(2),'c2 =',c2.round(2),perr[6].round(2),'w2 =',w2.round(2),perr[7].round(2)
print 'h3 =',h3.round(2),perr[8].round(2),'t3 =',t3.round(2),perr[9].round(2),'c3 =',c3.round(2),perr[10].round(2),'w3 =',w3.round(2),perr[11].round(2)
print 'hw =',hw.round(2),perr[12].round(2),'fw =',fw,'cw =',cw.round(2),perr[13].round(2),'ww =',ww.round(2),perr[14].round(2)
print 't0 =',t0.round(2),perr[15].round(2),'c0 =',c0.round(2),perr[16].round(2),'w0 =',w0.round(2),perr[17].round(2)


tau0 = t0*gaussian(vel, c0, w0)
tau1 = t1*gaussian(vel, c1, w1)
tau2 = t2*gaussian(vel, c2, w2)
tau3 = t3*gaussian(vel, c3, w3)
tau = tau0+tau1+tau2+tau3
Tc1 = h1*(1.-np.exp(-tau1))*np.exp(-tau0)
Tc2 = h2*(1.-np.exp(-tau2))*np.exp(-tau1)*np.exp(-tau0)
Tc3 = h3*(1.-np.exp(-tau3))*np.exp(-tau2)*np.exp(-tau1)*np.exp(-tau0)
Tc = Tc1+Tc2+Tc3
Tw = (fw+(1.-fw)*np.exp(-tau))*hw*gaussian(vel, cw, ww)

vmin = -15
vmax = 20

ax2 = fig.add_subplot(3,1,1)
plt.xlim(vmin,vmax)
ax2.plot(vel,stk_fit,'k-', label="I$_{obs}$")
ax2.plot(vel,Tc+Tw,'k:', label="T$_{fit}$")
ax2.plot(vel,Tw,'r:')
ax2.plot(vel,Tc1,'b--')
ax2.plot(vel,Tc2,'b-.')
ax2.plot(vel,Tc3,'b:')
plt.legend(loc=1,frameon=False,prop={'size':10})
plt.ylabel('Stokes I (K)')


[5.50169313 0.11676948 0.11004136 0.04645013 1.78569899 0.08169822
 0.33226137 0.13182842 2.03619027 0.95178143 0.08955034 0.06179257
 2.4724035  0.0406404  0.0858197  0.00651767 0.00669077 0.0086885 ]
h1 = 90.3163417633623 5.501693133401998 t1 = 0.831788135178355 0.11676947825919647 c1 = 8.121416508966051 0.11004136182642507 w1 = 1.8599794844009654 0.04645013291739349
h2 = 116.3150415619572 1.7856989850733587 t2 = 0.835445944697094 0.08169821682531658 c2 = -0.3886598611707412 0.3322613738880975 w2 = 2.4138301925883936 0.13182842216596435
h3 = 135.29332839735076 2.0361902660396245 t3 = 10.443153714306261 0.9517814254387347 c3 = 4.3828370809657855 0.08955033953567983 w3 = 2.040896307398208 0.06179256741548471
hw = 46.69468766097354 2.4724034985540344 fw = 0.0 cw = 2.625914025692626 0.040640399952519266 ww = 6.443224235730585 0.08581970007877136
t0 = 0.31675925493586315 0.006517669446865265 c0 = 6.970105884145499 0.006690773397044763 w0 = 0.40059151218685474 0.008688496951455859
h1 = 90.

Text(0,0.5,'Stokes I (K)')

In [17]:
### fig = plt.figure() 
#fit Blos, leakage

nrchnls = 497
zf = 2.8 #zeeman splitting factor of HI = 2.8 Hz/micronG
dvdf = 2.9979e5/1420.40575e6 #km/s to Hz factor

b0 = -0.01 #B0/((vel[1]-vel[0])/dvdf*2/zf)
b1 = -0.01 #B1/((vel[1]-vel[0])/dvdf*2/zf)
b2 = 0.01 #B2/((vel[1]-vel[0])/dvdf*2/zf)
b3 = 0.01 #B2/((vel[1]-vel[0])/dvdf*2/zf)
bw = 0.01
e = 1.4e-4

def fitv(vel, b0, b1, b2, b3, bw, e):
    RCP_tau0 = t0*gaussian(vel, c0+b0, w0)
    LCP_tau0 = t0*gaussian(vel, c0-b0, w0)
    RCP_tau1 = t1*gaussian(vel, c1+b1, w1)
    LCP_tau1 = t1*gaussian(vel, c1-b1, w1)
    RCP_tau2 = t2*gaussian(vel, c2+b2, w2)
    LCP_tau2 = t2*gaussian(vel, c2-b2, w2)
    RCP_tau3 = t3*gaussian(vel, c3+b3, w3)
    LCP_tau3 = t3*gaussian(vel, c3-b3, w3)
    RCP_tau = RCP_tau0+RCP_tau1+RCP_tau2+RCP_tau3
    LCP_tau = LCP_tau0+LCP_tau1+LCP_tau2+LCP_tau3

    RCP_Tc1 = h1/2*(1-np.exp(-RCP_tau1))*np.exp(-RCP_tau0)
    LCP_Tc1 = h1/2*(1-np.exp(-LCP_tau1))*np.exp(-LCP_tau0)
    RCP_Tc2 = h2/2*(1-np.exp(-RCP_tau2))*np.exp(-RCP_tau1)*np.exp(-RCP_tau0)
    LCP_Tc2 = h2/2*(1-np.exp(-LCP_tau2))*np.exp(-LCP_tau1)*np.exp(-LCP_tau0)
    RCP_Tc3 = h3/2*(1-np.exp(-RCP_tau3))*np.exp(-RCP_tau2)*np.exp(-RCP_tau1)*np.exp(-RCP_tau0)
    LCP_Tc3 = h3/2*(1-np.exp(-LCP_tau3))*np.exp(-LCP_tau2)*np.exp(-LCP_tau1)*np.exp(-LCP_tau0)

    RCP_Tw = (fw+(1.-fw)*np.exp(-RCP_tau))*hw/2*gaussian(vel, cw+bw, ww)
    LCP_Tw = (fw+(1.-fw)*np.exp(-LCP_tau))*hw/2*gaussian(vel, cw-bw, ww)
    RCP = RCP_Tc1+RCP_Tc2+RCP_Tc3+RCP_Tw
    LCP = LCP_Tc1+LCP_Tc2+LCP_Tc3+LCP_Tw
    model = RCP-LCP + e*stk_fit
    return model

b0b = [-0.1,0.1]
b1b = [-0.1,0.1]
b2b = [-0.1,0.1]
b3b = [-0.1,0.1]
bwb = [-0.1,0.1]
eb  = [-1.e-3,1.e-3]
from scipy.optimize import curve_fit
popt,pcov = curve_fit(fitv,vel,stk_Vfit,p0=[b0, b1, b2, b3, bw, e],
                      bounds=([b0b[0], b1b[0], b2b[0], b3b[0], bwb[0], eb[0]],
                              [b0b[1], b1b[1], b2b[1], b3b[1], bwb[1], eb[1]]))

print 'b0 =',popt[0],'b1 =',popt[1],'b2 =',popt[2],'b3 =',popt[3],'bw =',popt[4],'e =',popt[5]
print np.shape(pcov)
perr = np.sqrt(np.diag(pcov))
print perr

b0 = popt[0]
b1 = popt[1]
b2 = popt[2]
b3 = popt[3]
bw = popt[4]
e =  popt[5]

B0 = b0/dvdf*2/zf
B1 = b1/dvdf*2/zf
B2 = b2/dvdf*2/zf
B3 = b3/dvdf*2/zf
Bw = bw/dvdf*2/zf
B0err = perr[0]/dvdf*2/zf
B1err = perr[1]/dvdf*2/zf
B2err = perr[2]/dvdf*2/zf
B3err = perr[3]/dvdf*2/zf
Bwerr = perr[4]/dvdf*2/zf
print B0,B1,B2,B3,Bw
print B0err,B1err,B2err,B3err,Bwerr

RCP_tau0 = t0*gaussian(vel, c0+b0, w0)
LCP_tau0 = t0*gaussian(vel, c0-b0, w0)
RCP_tau1 = t1*gaussian(vel, c1+b1, w1)
LCP_tau1 = t1*gaussian(vel, c1-b1, w1)
RCP_tau2 = t2*gaussian(vel, c2+b2, w2)
LCP_tau2 = t2*gaussian(vel, c2-b2, w2)
RCP_tau3 = t3*gaussian(vel, c3+b3, w3)
LCP_tau3 = t3*gaussian(vel, c3-b3, w3)
RCP_tau = RCP_tau0+RCP_tau1+RCP_tau2+RCP_tau3
LCP_tau = LCP_tau0+LCP_tau1+LCP_tau2+LCP_tau3
    
RCP_Tc1 = h1/2*(1-np.exp(-RCP_tau1))*np.exp(-RCP_tau0)
LCP_Tc1 = h1/2*(1-np.exp(-LCP_tau1))*np.exp(-LCP_tau0)
RCP_Tc2 = h2/2*(1-np.exp(-RCP_tau2))*np.exp(-RCP_tau1)*np.exp(-RCP_tau0)
LCP_Tc2 = h2/2*(1-np.exp(-LCP_tau2))*np.exp(-LCP_tau1)*np.exp(-LCP_tau0)
RCP_Tc3 = h3/2*(1-np.exp(-RCP_tau3))*np.exp(-RCP_tau2)*np.exp(-RCP_tau1)*np.exp(-RCP_tau0)
LCP_Tc3 = h3/2*(1-np.exp(-LCP_tau3))*np.exp(-LCP_tau2)*np.exp(-LCP_tau1)*np.exp(-LCP_tau0)

RCP_Tw = (fw+(1.-fw)*np.exp(-RCP_tau))*hw/2*gaussian(vel, cw+bw, ww)
LCP_Tw = (fw+(1.-fw)*np.exp(-LCP_tau))*hw/2*gaussian(vel, cw-bw, ww)
RCP = RCP_Tc1+RCP_Tc2+RCP_Tc3+RCP_Tw
LCP = LCP_Tc1+LCP_Tc2+LCP_Tc3+LCP_Tw

ax4 = fig.add_subplot(3,1,2)
plt.ylim(-0.08,0.08)
plt.xlim(vmin,vmax)
ax4.step(vel,stk_Vfit-e*stk_fit,'k-', label=r"data")
ax4.plot(vel,RCP - LCP,'r-')
ax4.plot(vel,RCP_Tc1 - LCP_Tc1,'b--')
ax4.plot(vel,RCP_Tc2 - LCP_Tc2,'b-.')
ax4.plot(vel,RCP_Tc3 - LCP_Tc3,'b:')
ax4.plot(vel,RCP_Tw - LCP_Tw,'r:')
plt.ylabel('Stokes V (K)')

ax4 = fig.add_subplot(3,1,3)
plt.ylim(-0.08,0.08)
ax4.step(vel,stk_Vfit-e*stk_fit - (RCP - LCP),'k-', label=r"data")
ax4.plot([-15,20],[0,0],'k-')
plt.xlim(vmin,vmax)
plt.xlabel(r'V$_{lsr}$ (km/s)')

plt.savefig('fit_zeeman_method2_final.pdf')


b0 = -0.0011160444502391067 b1 = -0.0011727136570163165 b2 = 0.002255014261220494 b3 = -0.0008612508260375075 bw = 0.0008764056054687094 e = 0.0003447985562038476
(6, 6)
[9.61659923e-05 3.24636854e-04 2.87058533e-04 1.26971463e-04
 5.02279366e-04 1.39816188e-05]
-3.7770152306024123 -3.968800116103901 7.631616471934144 -2.9147203649600564 2.966008494851125
0.32545336127153696 1.0986644316761898 0.9714885914032019 0.42970792874726005 1.699857757329295


In [18]:
#plot figure 2

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
import numpy as np
import sys

font = {'family':'normal', 
        'weight':'normal',  
        'size'  :10}  
matplotlib.rc('font', **font)

fig = plt.figure(figsize=(6,8)) 
fig.subplots_adjust(0.15,0.15,0.85,0.85,0,0)

vmin = -7#-10
vmax = 15#20
ax2 = fig.add_subplot(2,1,1)
plt.ylim(0., 140.)
plt.xlim(vmin,vmax)
ax2.plot(vel,stk_fit,'k-', label="I$_{obs}$",linewidth=1.)
ax2.plot(vel,Tc+Tw,'k--', label="I$_{fit}$",linewidth=1.)
ax2.plot(vel,h1*(1.-np.exp(-tau1))+h3*(1.-np.exp(-tau3))*np.exp(-tau2)*np.exp(-tau1)-Tc1-Tc3,'r', label="HINSA")
ax2.plot(vel,h1*(1.-np.exp(-tau1)),'b--', label="HI, CNM",linewidth=1.)
ax2.plot(vel,Tc2,'b--',linewidth=1.)
ax2.plot(vel,h3*(1.-np.exp(-tau3))*np.exp(-tau2)*np.exp(-tau1),'b--',linewidth=1.)
ax2.plot(vel,Tw,'b:', label="HI, WNM",linewidth=1.)

plt.legend(loc=1,frameon=False,prop={'size':10})
plt.ylabel('Stokes I (K)')
ax2.set_xticklabels([])
xminor = MultipleLocator(1)
yminor = MultipleLocator(10)
ax2.xaxis.set_minor_locator(xminor)
ax2.yaxis.set_minor_locator(yminor)
ax2.xaxis.set_major_locator(MultipleLocator(5))
ax2.tick_params(which='both',right=True,top=True)
ax2.text(0.06,1-0.06*8/6, 'a',horizontalalignment='center',verticalalignment='center',
         transform=ax2.transAxes, fontsize=15,fontweight='bold')

ax5 = fig.add_subplot(2,1,2)
plt.ylim(-0.09,0.07)
plt.xlim(vmin,vmax)
ax5.step(vel,stk_Vfit-e*stk_fit,'k-', label=r"V$_{obs}$",linewidth=1.)
ax5.plot(vel,RCP - LCP,'k--', label=r"V$_{fit}$",linewidth=1.)
ax5.plot(vel, HINSAV,'r-',label="HINSA")

plt.legend(loc=1,frameon=False,prop={'size':10})
plt.xlabel(r'LSR Velocity (km s$^{-1}$)')
plt.ylabel('Stokes V (K)')
xminor = MultipleLocator(1)
yminor = MultipleLocator(0.01)
ax5.xaxis.set_minor_locator(xminor)
ax5.yaxis.set_minor_locator(yminor)
ax5.xaxis.set_major_locator(MultipleLocator(5))
ax5.tick_params(which='both',right=True,top=True)
ax5.text(0.06,1-0.06*8/6, 'b',horizontalalignment='center',verticalalignment='center',
         transform=ax5.transAxes, fontsize=15,fontweight='bold')

plt.savefig('fit_zeeman_method2_wcca_gttt11_paper.pdf')

b = np.array([vel,stk_fit,Tc+Tw,h1*(1.-np.exp(-tau1))+h3*(1.-np.exp(-tau3))*np.exp(-tau2)*np.exp(-tau1)-Tc1-Tc3,
              h1*(1.-np.exp(-tau1)),Tc2,h3*(1.-np.exp(-tau3))*np.exp(-tau2)*np.exp(-tau1),Tw,
              stk_Vfit-e*stk_fit,RCP - LCP,HINSAV])
np.savetxt('fig2.csv', np.transpose(b), header="velocity (km/s),I_data (K),I_fit (K),HINSA_I (K),CNM1_I (K)\
           ,CNM2_I (K),CNM3_I (K),WNM_I (K),V_data (K),V_fit (K),HINSA_V (K)",delimiter=',',fmt="%f", comments='')


In [21]:
#plot figure 3

from matplotlib.ticker import MultipleLocator, FormatStrFormatter
fig = plt.figure(figsize=(6,8)) 
fig.subplots_adjust(0.15,0.15,0.85,0.85,0,0)

import matplotlib as mpl
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'

Tc1V = h1/2*(1-np.exp(-RCP_tau1)) - h1/2*(1-np.exp(-LCP_tau1))
hinsaV = RCP_Tc1 - LCP_Tc1 - Tc1V
Tc2V = RCP_Tc2 - LCP_Tc2
Tc3V = RCP_Tc3 - LCP_Tc3
TwV = RCP_Tw - LCP_Tw
resV = stk_Vfit - e*stk_fit -(RCP-LCP)

Tc1V_noHINSA = h1/2*(1-np.exp(-RCP_tau1)) - h1/2*(1-np.exp(-LCP_tau1))
Tc2V_noHINSA = h2/2*(1-np.exp(-RCP_tau2))*np.exp(-RCP_tau1) - h2/2*(1-np.exp(-LCP_tau2))*np.exp(-LCP_tau1)
Tc3V_noHINSA = h3/2*(1-np.exp(-RCP_tau3))*np.exp(-RCP_tau2)*np.exp(-RCP_tau1) - h3/2*(1-np.exp(-LCP_tau3))*np.exp(-LCP_tau2)*np.exp(-LCP_tau1)
TwV_noHINSA = np.exp(-RCP_tau1-RCP_tau2-RCP_tau3)*hw/2*gaussian(vel, cw+bw, ww)-np.exp(-LCP_tau1-LCP_tau2-LCP_tau3)*hw/2*gaussian(vel, cw-bw, ww)
HINSAV = (RCP-LCP)-Tc1V_noHINSA-Tc2V_noHINSA-Tc3V_noHINSA -TwV_noHINSA

vmin2 = -7
vmax2 = 15
ymin = -0.12
ymax = 0.12
xminor = MultipleLocator(1)
yminor = MultipleLocator(0.01)

ax2 = fig.add_subplot(5,1,1)
plt.ylim(-0.08,0.08)
plt.xlim(vmin2,vmax2)
ax2.plot(vel, HINSAV,'r-')
ax2.step(vel, resV+HINSAV,'k-', label=r"data")
ax2.yaxis.set_major_locator(MultipleLocator(0.05))
ax2.xaxis.set_minor_locator(xminor)
ax2.yaxis.set_minor_locator(yminor)
ax2.set_xticklabels([])
ax2.text(0.08,1-0.15*8/6, 'HINSA',horizontalalignment='center',verticalalignment='center',
         transform=ax2.transAxes, fontsize=10)
ax2.text(0.25,1-0.15*8/6, r'$+$3.8$\pm$0.3 $\mu$G',horizontalalignment='center',verticalalignment='center',
         transform=ax2.transAxes, fontsize=10)

ax2 = fig.add_subplot(5,1,2)
plt.ylim(-0.08,0.08)
#plt.ylim(ymin,ymax)
plt.xlim(vmin2,vmax2)
ax2.plot(vel, Tc1V_noHINSA,'r-')
ax2.step(vel, resV+Tc1V_noHINSA,'k-', label=r"data")
ax2.yaxis.set_major_locator(MultipleLocator(0.05))
ax2.xaxis.set_minor_locator(xminor)
ax2.yaxis.set_minor_locator(yminor)
ax2.set_xticklabels([])
ax2.text(0.08,1-0.15*8/6, 'CNM1',horizontalalignment='center',verticalalignment='center',
         transform=ax2.transAxes, fontsize=10)
ax2.text(0.25,1-0.15*8/6, r'$+$4.0$\pm$1.1 $\mu$G',horizontalalignment='center',verticalalignment='center',
         transform=ax2.transAxes, fontsize=10)

ax2 = fig.add_subplot(5,1,3)
plt.ylim(-0.06,0.10)
#plt.ylim(ymin,ymax)
plt.xlim(vmin2,vmax2)
ax2.plot(vel, Tc2V_noHINSA,'r-')
ax2.step(vel, resV+Tc2V_noHINSA,'k-', label=r"data")
plt.ylabel('Stokes V (K)')
ax2.yaxis.set_major_locator(MultipleLocator(0.05))
ax2.xaxis.set_minor_locator(xminor)
ax2.yaxis.set_minor_locator(yminor)
ax2.set_xticklabels([])
ax2.text(0.08,1-0.15*8/6, 'CNM2',horizontalalignment='center',verticalalignment='center',
         transform=ax2.transAxes, fontsize=10)
ax2.text(0.25,1-0.15*8/6, r'$-$7.6$\pm$1.0 $\mu$G',horizontalalignment='center',verticalalignment='center',
         transform=ax2.transAxes, fontsize=10)

ax2 = fig.add_subplot(5,1,4)
plt.ylim(-0.10,0.06)
#plt.ylim(ymin,ymax)
ax2.plot(vel, Tc3V_noHINSA,'r-')
ax2.step(vel, resV+Tc3V_noHINSA,'k-', label=r"data")
plt.xlim(vmin2,vmax2)
ax2.yaxis.set_major_locator(MultipleLocator(0.05))
ax2.xaxis.set_minor_locator(xminor)
ax2.yaxis.set_minor_locator(yminor)
ax2.set_xticklabels([])
ax2.text(0.08,0.15*8/6, 'CNM3',horizontalalignment='center',verticalalignment='center',
         transform=ax2.transAxes, fontsize=10)
ax2.text(0.25,0.15*8/6, r'$+$2.9$\pm$0.4 $\mu$G',horizontalalignment='center',verticalalignment='center',
         transform=ax2.transAxes, fontsize=10)

ax2 = fig.add_subplot(5,1,5)
plt.ylim(-0.08,0.08)
#plt.ylim(ymin,ymax)
ax2.plot(vel, TwV_noHINSA,'r-')
ax2.step(vel, resV+TwV_noHINSA,'k-', label=r"data")
plt.xlim(vmin2,vmax2)
ax2.yaxis.set_major_locator(MultipleLocator(0.05))
ax2.xaxis.set_minor_locator(xminor)
ax2.yaxis.set_minor_locator(yminor)
plt.xlabel(r'LSR Velocity (km s$^{-1}$)')
ax2.text(0.08,1-0.15*8/6, 'WNM',horizontalalignment='center',verticalalignment='center',
         transform=ax2.transAxes, fontsize=10)
ax2.text(0.25,1-0.15*8/6, r'$-$3.0$\pm$1.7 $\mu$G',horizontalalignment='center',verticalalignment='center',
         transform=ax2.transAxes, fontsize=10)

plt.savefig('fit_zeeman_method2_wcca_gttt11_2.pdf')

a = np.array([vel,resV+HINSAV,HINSAV,resV+Tc1V_noHINSA,Tc1V_noHINSA,resV+Tc2V_noHINSA,Tc2V_noHINSA,
              resV+Tc3V_noHINSA,Tc3V_noHINSA,resV+TwV_noHINSA,TwV_noHINSA])
np.savetxt('fig3.csv', np.transpose(a), header="velocity (km/s),HINSAV_data (K),HINSAV_fit (K),CNM1_data (K),CNM1_fit (K),\
           CNM2_data (K),CNM2_fit (K),CNM3_data (K),CNM3_fit (K),WNM_data (K),WNM_fit (K)",delimiter=',',fmt="%f", comments='')
